In [1]:
# IMPORT STATEMENTS

import argparse
import confluent_kafka
from uuid import uuid4
from six.moves import input
from confluent_kafka import Producer
from confluent_kafka.serialization import StringSerializer, SerializationContext, MessageField
from confluent_kafka.schema_registry import SchemaRegistryClient,Schema
from confluent_kafka.schema_registry.json_schema import JSONSerializer
#from confluent_kafka.schema_registry import *
import pandas as pd
from typing import List

In [2]:
#CONFIGURATION DETAILS

FILE_PATH = r"C:\Users\HP\Desktop\HARIKRISHNAN_DETAILS\DATA ENGINEERING\KAFKA\restaurant_orders.csv"

columns=['Order Number', 'Order Date', 'Item Name', 'Quantity', 'Product Price', 'Total products']

API_KEY = 'LAMZT52YQ3KXOLCK'
ENDPOINT_SCHEMA_URL  = 'https://psrc-nx65v.us-east-2.aws.confluent.cloud'
API_SECRET_KEY = 'M9R2ELFp/CYUUc56U1H1vbsKyse1EtCNrR3qqGUpQ7F72gCRT62Lc5+jWpLid1oB'
BOOTSTRAP_SERVER = 'pkc-921jm.us-east-2.aws.confluent.cloud:9092'
SECURITY_PROTOCOL = 'SASL_SSL'
SSL_MACHENISM = 'PLAIN'
SCHEMA_REGISTRY_API_KEY = 'T6UDFASJDL3RKCWZ'
SCHEMA_REGISTRY_API_SECRET = 'KQNwg+/HbCglNBys5AeK542yYObZCV7IbYj1w0Fxo3E1sOQd3UZ1pzaTAYUQ+/Db'

In [3]:
#CONFIGURATION FUNCTIONS

def sasl_conf():

    sasl_conf = {'sasl.mechanism': SSL_MACHENISM,
                 # Set to SASL_SSL to enable TLS support.
                #  'security.protocol': 'SASL_PLAINTEXT'}
                'bootstrap.servers':BOOTSTRAP_SERVER,
                'security.protocol': SECURITY_PROTOCOL,
                'sasl.username': API_KEY,
                'sasl.password': API_SECRET_KEY
                }
    return sasl_conf



def schema_config():
    return {'url':ENDPOINT_SCHEMA_URL,
    
    'basic.auth.user.info':f"{SCHEMA_REGISTRY_API_KEY}:{SCHEMA_REGISTRY_API_SECRET}"

    }

In [4]:
# FUNCTION FOR READING THE DATA FROM FILE AND TO DICT

def get_restaurant_instance(file_path):
    df=pd.read_csv(file_path)
    restaurants =[]
    for data in df.values:
        restaurant= dict(zip(columns,data))
        restaurants.append(restaurant)
    return restaurants

In [5]:
# DELIVERY REPORT

def delivery_report(err, msg):
    """
    Reports the success or failure of a message delivery.
    Args:
        err (KafkaError): The error that occurred on None on success.
        msg (Message): The message that was produced or failed.
    """

    if err is not None:
        print("Delivery failed for User record {}: {}".format(msg.key(), err))
        return
    print('User record {} successfully produced to {} [{}] at offset {}'.format(
        msg.key(), msg.topic(), msg.partition(), msg.offset()))

In [6]:
def main(topic):
    schema_str = """
    {
  "$id": "http://example.com/myURI.schema.json",
  "$schema": "http://json-schema.org/draft-07/schema#",
  "additionalProperties": false,
  "description": "Sample schema to help you get started.",
  "properties": {
    "Order Number": {
      "description": "The type(v) type is used.",
      "type": "number"
    },
    "Order Date": {
      "description": "The type(v) type is used.",
      "type": "string"
    },
    "Item Name": {
      "description": "The type(v) type is used.",
      "type": "string"
    },
    "Quantity": {
      "description": "The type(v) type is used.",
      "type": "number"
    },
    "Product Price": {
      "description": "The type(v) type is used.",
      "type": "number"
    },
    "Total products": {
      "description": "The type(v) type is used.",
      "type": "number"
    }
  },
  "title": "SampleRecord",
  "type": "object"
}
    """
    schema_registry_conf = schema_config()
    schema_registry_client = SchemaRegistryClient(schema_registry_conf)
    schema = Schema(schema_str, 'JSON')
    print(schema_registry_client.register_schema('restaurants-value', schema, normalize_schemas=False))
    latest_version = schema_registry_client.get_latest_version('restaurants-value')
    string_serializer = StringSerializer()
    json_serializer = JSONSerializer(latest_version.schema, schema_registry_client)

    producer = Producer(sasl_conf())

    print("Producing user records to topic {}. ^C to exit.".format("restaurants-value-data"))

    producer.poll(0.0)

    try:
        restaurants = get_restaurant_instance(file_path=FILE_PATH)
        for restaurant in restaurants:

#             print(case)
            producer.produce(topic=topic,
                            key=string_serializer(str(uuid4())),
                            value=json_serializer(restaurant, SerializationContext(topic, MessageField.VALUE)),
                            on_delivery=delivery_report)
#             break
    except KeyboardInterrupt:
        pass
    except ValueError:
        print("Invalid input, discarding record...")
        pass

    print("\nFlushing records...")
    producer.flush()

In [7]:
main("restaurants")

100011
Producing user records to topic restaurants-value-data. ^C to exit.


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

